# Problem 41: Pandigital prime
We shall say that an n-digit number is pandigital if it makes use of all the digits 1 to n exactly once. For example, 2143 is a 4-digit pandigital and is also prime.

What is the largest n-digit pandigital prime that exists?

In [1]:
import string
from numpy import gcd

In [2]:
def descendingPerms(elements, already_sorted=False):
    # generator of all permutations of elements in descending order 
    # (permutations will not be unique if each element not unique)
    if not elements:
        yield elements[0:0]
    else:
        if not already_sorted:
            s = sorted(elements)
            elements = ''.join(s) if isinstance(elements, str) else s
        for i in reversed(range(len(elements))):
            el = elements[i:i+1]
            no_el = elements[:i] + elements[i+1:]
            for perm in descendingPerms(no_el, already_sorted=True):
                yield el + perm

In [3]:
def isPrime(N):
    # True if N is prime, else False
    if N < 2: return False
    root = int(N**0.5)
    for i in range(2, root+1):
        if N%i == 0:
            return False
    return True

In [4]:
def pandigitalPrime(n):
    # largest n-digit pandigital prime
    base = 10
    digits = string.digits[1:n+1]
    not_last = []
    for d in digits:
        if gcd(base, int(d)) > 1:
            not_last += d
    for perm in descendingPerms(digits):
        if perm[-1] in not_last:
            continue
        elif isPrime(int(perm)):
            return int(perm)
    else:
        return 0

In [5]:
def largestPandigitalPrime():
    # largest base 10 pandigital prime
    base = 10
    for i in reversed(range(2, base)):
        pand = pandigitalPrime(i)
        if pand > 0:
            return pand

In [6]:
largestPandigitalPrime()
# should return 7652413

7652413

# Problem 42: Coded triangle numbers
The nth term of the sequence of triangle numbers is given by, $t_n = n(n+1)/2$; so the first ten triangle numbers are:

$1, 3, 6, 10, 15, 21, 28, 36, 45, 55, ...$

By converting each letter in a word to a number corresponding to its alphabetical position and adding these values we form a word value. For example, the word value for SKY is $19 + 11 + 25 = 55 = t_{10}$. If the word value is a triangle number then we shall call the word a triangle word.

Using data > words.txt, a 16K text file containing nearly two-thousand common English words, how many are triangle words?

In [7]:
# load names
import numpy as np
from pathlib import Path
wordsFile = Path('data') / 'words.txt'
words = np.loadtxt(wordsFile, delimiter=',', dtype='str')
words = [w.replace('"', '') for w in words]

In [8]:
def codedTriangleNumbers(word_array):
    # number of triangle-number-valued words in word_array
    offset = 64
    tot = 0
    for word in word_array:
        val = sum(ord(c) - offset for c in word)
        triangle_position = (8 * val + 1)**0.5
        if triangle_position.is_integer():
            tot += 1
    return tot

In [9]:
codedTriangleNumbers(words)
# should return 162

162

# Problem 43: Sub-string divisibility

The number, 1406357289, is a 0 to 9 pandigital number because it is made up of each of the digits 0 to 9 in some order, but it also has a rather interesting sub-string divisibility property.

Let d1 be the 1st digit, d2 be the 2nd digit, and so on. In this way, we note the following:

$d_2d_3d_4=406$ is divisible by 2

$d_3d_4d_5=063$ is divisible by 3

$d_4d_5d_6=635$ is divisible by 5

$d_5d_6d_7=357$ is divisible by 7

$d_6d_7d_8=572$ is divisible by 11

$d_7d_8d_9=728$ is divisible by 13

$d_8d_9d_10=289$ is divisible by 17

Find the sum of all 0 to 9 pandigital numbers with this property.

In [10]:
import string

In [11]:
def substringDivisibility():
    # sum of all pandigital numbers with the property described above
    base = 10
    all_digits = string.digits
    primes = [2, 3, 5, 7, 11, 13, 17]
    
    pandigitals = []
    for i in range(0, base**3, primes[-1]):
        num_str = str(i).zfill(3)
        digits = all_digits
        potential_candidate = True
        
        for d in num_str:
            if d in digits:
                digits = digits.replace(d, '')
            else:
                potential_candidate = False
                break
                
        if not potential_candidate:
            continue
            
        candidates = [num_str]
        for p in reversed(primes[:-1]):
            new_candidates = []
            for cand in candidates:
                rem_digits = digits
                for d in cand:
                    rem_digits = rem_digits.replace(d, '')
                    
                for d in rem_digits:
                    num_str = d + cand[:2]
                    if int(num_str)%p == 0:
                        if len(rem_digits) == 2:
                            last_digit = rem_digits.replace(d, '')
                            if last_digit == '0':
                                continue
                            d = rem_digits.replace(d, '') + d
                        new_candidates += [d + cand]
                        
            candidates = new_candidates
            if not candidates:
                break
        else:
            pandigitals += candidates
            
    return sum(int(pand) for pand in pandigitals)

In [12]:
substringDivisibility()
# should return 16695334890

16695334890

# Problem 44: Pentagon numbers
Pentagonal numbers are generated by the formula, $P_n=n(3n−1)/2$. The first ten pentagonal numbers are:

$1, 5, 12, 22, 35, 51, 70, 92, 117, 145, ...$

It can be seen that $P_4 + P_7 = 22 + 70 = 92 = P_8$. However, their difference, $70 − 22 = 48$, is not pentagonal.

Find the pair of pentagonal numbers, $P_j$ and $P_k$, for which their sum and difference are pentagonal and $D = |Pk − Pj|$ is minimised; what is the value of $D$?

In [13]:
import itertools

In [14]:
def P(n):
    # return nth pentagonal number
    return n * (3*n - 1) // 2

In [15]:
def pentagon_differences(N):
    # return first N pentagonal differences D which satisfy conditions
    # of problem 44
    Ds = []
    
    # iterate over x = 5 mod 6
    for x in itertools.count(5, 6):
        q = (x**2 - 1) // 12
        
        # iterate over factors of q
        root_q = int(q**0.5)
        for n in range(2, root_q + 1):
            if q%n != 0:
                continue
                
            # check condition 1    
            large_factor = q // n
            s = (large_factor + 1) / 3
            condition_1 = ((s - n) % 2 == 0) and (s > n)
            if not condition_1:
                continue
            
            # check condition 2
            b_discriminant = 1 + 18*(s**2 + n**2) - 12*s
            root_disc = b_discriminant**0.5
            condition_2 = (root_disc%6 == 5)
            if not condition_2:
                continue
            
            # calculate a, D and related pentagonal numbers
            a = (1 + x) // 6
            D = P(a)
            Ds += [D]
            b = int((1 + root_disc) / 6)
            k = int((s - n) / 2)
            # uncomment to print related pentagonal numbers
            #print('D = %s' % D)
            #print('[k = %s, n = %s, a = %s, b = %s, n mod 6 = %s,' \
            #      % (k, n, a, b, n%6))
            #print(' P_k = %s, P_(k+n) = %s, P_b = %s]\n' \
            #      % (P(k), P(k+n), P(b)))
            break
            
        if len(Ds) == N:
            break
            
    return Ds

In [16]:
pentagon_differences(1)[0]
# should return 5482660

5482660

# Problem 45: Triangular, pentagonal, and hexagonal

Triangle, pentagonal, and hexagonal numbers are generated by the following formulae:

Triangle $T_n=n(n+1)/2$

$1, 3, 6, 10, 15, ...$

Pentagonal $P_n=n(3n−1)/2$

$1, 5, 12, 22, 35, ...$

Hexagonal $H_n=n(2n−1)$

$1, 6, 15, 28, 45, ...$

It can be verified that $T_{285} = P_{165} = H_{143} = 40755$.

Find the next triangle number that is also pentagonal and hexagonal.

In [17]:
def triPentaHexa(start, max_iter=10**7):
    # return first triangle number >= start 
    # that is also pentagonal and hexagonal
    start_h = (1 + (8*start + 1)**0.5) / 4
    if start_h.is_integer():
        start_h = int(start_h)
    else:
        start_h = int(start_h) + 1
        
    for h in range(start_h, max_iter):
        H = h * (2*h - 1)
        is_P = ((1 + (24*H + 1)**0.5) / 6).is_integer()
        if is_P:
            is_T = ((1 + (8*H + 1)**0.5) / 4).is_integer()
            if is_T:
                return H
        h += 1
    else:
        print('No tri-penta-hexa numbers found in range')
        return

In [18]:
triPentaHexa(40756)
# should return 1533776805

1533776805